In [1]:
import glob
import pandas as pd
import numpy as np
from data_generator.batch_generator import BatchGenerator
from squeezenet_master.model import *
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint#, TensorBoard
import pickle
from models import AlexNet, LeNet
from keras import backend as K
from math import sqrt
K.set_image_data_format('channels_first')

Using TensorFlow backend.


In [2]:
approach = 'abordagem-squeeze1' 
activation = 'relu'
net = 'squeeze'

In [3]:
test_dataset = BatchGenerator(box_output_format=['class_id'])
test_dataset.parse_csv(labels_filename='dataset/csv/imdb_csv/imdb_age_regression_test_split_47950-70-10-20.csv', 
                        images_dir='dataset/imdb-hand-crop/',
                        input_format=['image_name', 'class_id'])

In [4]:
img_height, img_width, img_depth = (224,224,3)

epochs = 90
melhor_epoca='37'
train_batch_size = 128
shuffle = True
ssd_train = False

validation_batch_size = 32


# In[15]:

test_generator = test_dataset.generate(batch_size=train_batch_size,
                                       shuffle=shuffle,
                                       ssd_train=ssd_train,
                                       divide_by_stddev = 225,
                                       #equalize=True,
                                       channels_first=True,
                                       returns={'processed_labels'},
                                       resize=(img_height, img_width))

In [5]:
print("Number of images in the dataset:", test_dataset.get_n_samples())

Number of images in the dataset: 9590


In [6]:
steps = test_dataset.get_n_samples()/train_batch_size

In [38]:
img_treat = approach
activation = 'relu'
net = 'squeeze'

if net == 'alexnet':
    model = AlexNet
elif net =='lenet':
    model = LeNet
elif net == 'vgg16':
    model = VGG16 
csvlogger_name = 'callbacks/'+net +'/age/consolidados/history-regression-' + approach + '-' + activation + '.csv'
checkpoint_filename = 'callbacks/'+net+'/age/consolidados/class-weights-' + approach + '-' + activation + '.{epoch:02d}-{val_loss:.2f}.hdf5'
csvlogger_name, checkpoint_filename

weights_path = glob.glob('callbacks/' + net + '/age/consolidados/class-weights-' + img_treat + '-' + activation + '*')[0]
#weights_path = glob.glob('callbacks/' + net + '/age/class-weights-' + img_treat + '-' + activation + '.' + melhor_epoca + '*')[0]

model = SqueezeNet(1, inputs=(img_depth, img_width, img_height))


model.load_weights(weights_path)

In [39]:
weights_path

'callbacks/squeeze/age/consolidados/class-weights-abordagem-squeeze1-relu.37-10.75.hdf5'

In [40]:
optimizer = Adam(lr=0.0000001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0000000001, amsgrad=True)

In [41]:
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

In [42]:
stats = model.evaluate_generator(test_generator,steps=30)


sqrt(stats[0]), stats

(13.846121028850387, [191.7150675455729, 10.7217347462972])

In [43]:
start = weights_path.find(activation)+len(activation)+1
end = start + 2
epoca = int(weights_path[start:end]) + 1

In [44]:
int(weights_path[start:end]) + 1

38

In [59]:
try:
    df_results = pd.read_csv('graficos/results_teste.csv', index_col=0)

except FileNotFoundError:
    df_results = pd.DataFrame({'Rede':[], 'Ativação':[], 'RMSE':[], 'MAE':[], 'FASE': [], 'Epoca': []})
df_results = df_results.append({'Rede':net, 'Ativação':activation, 'RMSE': sqrt(stats[0]), 'MAE':stats[1], 'FASE': img_treat, 'Epoca': epoca}, ignore_index=True)

# In[13]:

#df_results.to_csv('graficos/results_teste_' + img_treat + '.csv', index_col=0)
df_results.to_csv('graficos/results_teste.csv')



In [70]:
df_results.to_csv('graficos/results_teste.csv')
df_results

,Ativação,FASE,MAE,RMSE,Rede,Epoca
0,relu,image-treat-1,10.537234,13.553820,lenet,4.0
1,lrelu,image-treat-1,38.330450,40.826534,lenet,8.0
2,relu,image-treat-1,11.032492,13.764649,alexnet,5.0
3,lrelu,image-treat-1,39.278125,41.974881,alexnet,5.0
4,relu,image-treat-2,37.854424,40.272234,lenet,39.0
5,lrelu,image-treat-2,38.501557,41.065953,lenet,21.0
6,relu,image-treat-2,11.591327,14.595028,alexnet,16.0
7,lrelu,image-treat-2,28.062363,31.811022,alexnet,16.0
8,relu,image-treat-3,38.656497,41.200746,lenet,46.0
9,lrelu,image-treat-3,38.261533,40.855255,lenet,38.0


In [66]:
new_df_results =  df_results.copy()

In [67]:
new_df_results.drop(index=[i for i in range(16,20)], inplace=True)

In [68]:
df_results = new_df_results

In [47]:
prev = next(test_generator)


#prev[1]
predict = model.predict(prev[0])


In [49]:
print(predict[:10])

[[35.36179 ]
 [37.19529 ]
 [34.78997 ]
 [34.65884 ]
 [34.10476 ]
 [32.963337]
 [33.819687]
 [37.23399 ]
 [42.443604]
 [34.917072]]
